# Hill Cypher Encryption
---

In this example, we'll be using a linear transformation (and its inverse) to encode and decode a string of lowercase characters.

In [18]:
# importing numpy for linear algebra operations
import numpy as np

## Problem setup 1: (integer representation of the message string)
---
Our goal here will be to take the string "ibotta" and encrypt it via a linear transformation. To do this, we'll first map each character in the lowercase string to the integer range [0, 25] (where a = 0, b = 1, ..., z = 25). Here is the result of that mapping:

```
i -> 8
b -> 1
o -> 14
t -> 19
t -> 19
a -> 0
```

We'll use these values to populate an integer vector representation of the string. I'm going to initialize a (6,1) numpy matrix to do this. Note the transpose ('T') used to convert the default interpretation of a *row* vector/matrix to the *column* format we want.

In [16]:
messageIntVec = np.matrix([8, 1, 14, 19, 19, 0]).T

In [17]:
print(messageIntVec)

[[ 8]
 [ 1]
 [14]
 [19]
 [19]
 [ 0]]


## Problem setup 2: (transformation and inverse)
---
There are more general ways of constructing Hill cyphers/transformations than what we'll explore today (and you can read more at https://en.wikipedia.org/wiki/Hill_cipher). For our illustration, though, we'll use a simple composition that'll allow us to easily determine a forward (encryption) and inverse (decryption) transformation.

Let's say we want to use the secret key 'mykey' to encrypt the information. Here's one way we can do that.

First, as we did with the 'ibotta' string, let's transform the 'mykey' string into an integer array representation.

```
m -> 12
y -> 24
k -> 10
e -> 4
y -> 24
```

I'm actually going to keep these values as a *row* vector because of how they'll be subsequently used.

In [ ]:
A = np.matrix([[5, 1, 3],
             [1, 1, 1],
             [1, 2, 1]])

x = np.matrix([1,1,1]).T

A*x